In [ ]:
import torch

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from hf_token import hf_token

access_token = hf_token
login(token=access_token)

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=access_token)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=access_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pkk2125/.cache/huggingface/token
Login successful


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from utils.lora_ckpt import load_lora

In [9]:
from peft import LoraConfig, get_peft_model

In [8]:
lora_modules = ['q_proj', 'v_proj']
lora_r = 16
lora_alpha = lora_r
lora_dropout = 0.05


lora_config = LoraConfig(
        r = lora_r,
        lora_alpha = lora_alpha,
        target_modules = lora_modules,
        lora_dropout = lora_dropout,
        bias = "none"
)

In [10]:
lora_llm = get_peft_model(model=model, peft_config=lora_config)

In [11]:
load_lora(lora_llm, weights_path, end_of_epoch=None)

LoRA loaded from  ./weights/lora_llm.ckpt


In [12]:
def read_prompt(llm, tokenizer, prompt, device='cuda'):
    # Tokenize
    tokens = tokenizer(
        prompt, padding=True, return_tensors='pt'
    )['input_ids'].to(device)
    
    # Encode
    words_embed = llm(
        tokens, output_hidden_states=True
    ).hidden_states[-1] # last layer

    return words_embed[:, -1, :] # last or EOS token

In [15]:
tokenizer.pad_token = '[PAD]'

In [19]:
prompt = 'Remove male speaker'
embedding = read_prompt(lora_llm, tokenizer, prompt, device='cpu')

In [21]:
print(embedding.shape)

torch.Size([1, 4096])
